In [2]:
import pandas as pd
import numpy as np
import datetime
import requests
from string import Template
import pytz
import math
import logging
import time
from pandasql import sqldf
import re
import sqlite3

In [3]:
pd.options.display.max_rows = 50
pd.options.display.max_columns = 100

In [4]:
options_df = pd.read_pickle('0-2DTE_spy_options_01Jan23-28Feb23v2.pkl')
mysqldf = lambda q: sqldf(q, globals())
options_df['options_pct_change'] = options_df['options_pct_change'].astype(float)
options_df['equity_pct_change_normalized'] = (options_df['equity_pct_change']-1)*100

# day classification for ranges. below range boundaries are in absolute pct. Ex: 1 - 1%
very_high_range = 2
high_range = 1
average_high_range = 0.8
average = 0.5

options_df['day_classification'] = None
for day in options_df['time_converted'].dt.date.unique():
    day_mask = options_df['time_converted'].dt.date == day
    #find the maximum abs pct change
    max_pct_daily = max(options_df.loc[day_mask,'equity_pct_change_normalized'].max(),
                        abs(options_df.loc[day_mask,'equity_pct_change_normalized'].min()))
    if max_pct_daily > very_high_range:
        options_df.loc[day_mask,'day_classification'] = 'very_high'
    elif max_pct_daily > high_range:
        options_df.loc[day_mask,'day_classification'] = 'high'
    elif max_pct_daily > average_high_range:
        options_df.loc[day_mask,'day_classification'] = 'average_high'
    elif max_pct_daily > average:
         options_df.loc[day_mask,'day_classification'] = 'average'
    else:
         options_df.loc[day_mask,'day_classification'] = 'very_low'

#calculate DTE and convert to days resulting timedelta object
options_df['DTE'] = (options_df['expiry'] - options_df['time_converted'].dt.date) # returns timedelta
options_df['DTE'] = options_df['DTE'].apply(lambda timedeltaDTE: timedeltaDTE.days) # flip to days


# also add day of week for potential future analysis
options_df['day_name'] = options_df['time_converted'].dt.day_name()


# because of sat/sunday - we have now 3 and 4 DTE
# for now create new column DTE_adjusted where DTE  3 is DTE 1 (since Mon to Friday)
# and DTE 4 becomes DTE 2 (Thu=>Mon/Fri=>Tue)
# small UDF for it:
def _dte_adjusted_func(DTE):
    if DTE == 3: return 1
    elif DTE == 4: return 2
    else: return DTE
options_df['DTE_adjusted'] = options_df['DTE'].apply(_dte_adjusted_func)


In [5]:
## DO not run this cell! - temp fix


# found a bug in the options_pull_quotes where expiry date was wrong due to parsing
# one time fix - just parse options full name to pull expiry date again and populate
#  this column
# options_df['expiry'] = pd.to_datetime(options_df['full_name'].apply(
#     lambda option_full_string: re.search(r'\d{6}', option_full_string).group()
#       if re.search(r'\d{6}', option_full_string) else None), format='%y%m%d')

# check
# options_df['expiry_test'].unique()
#options_df.to_pickle('0-2DTE_spy_options_01Jan23-28Feb23.pkl')

In [6]:
options_df.groupby(['day_classification'])['equity_pct_change_normalized'].agg(['min','max']).sort_values(['day_classification'])

,min,max
day_classification,,
average,-0.730867,0.797115
average_high,-0.995763,0.867525
high,-1.778552,1.793822
very_high,0.000000,2.331606
very_low,-0.496885,0.473105


In [7]:
# DO NOT RUN
#this stuff failed on lambda. UDF needs to get DF,while looks
#like it gets passed with series. so SQL below works much more easier

results = (options_df[options_df['options_earliest_open'] > 0.15]
             .groupby([pd.to_datetime(options_df['time_converted']).dt.date, 'DTE'])
             .agg(date=('time_converted', 'min'),
                   day_classification=('day_classification', 'first'),
                   max_options_pct_change=('options_pct_change', 'max'),
                   time_of_max_change=('time_converted', lambda x: x.loc[x['options_pct_change'].idxmax()])))

KeyError: 'options_pct_change'

In [ ]:
# tried to create this summary table in pandas, but failed
# so switching to SQL instead
# also using DTE adjusted

max_min_df = mysqldf("""
        
        WITH 
        
        MAX_TABLE AS (      
        select distinct date(time_converted) as date
        ,day_name
        , DTE_adjusted
        , day_classification
        , max(options_pct_change) as max_options_pct_change
        , time_converted as time_of_max_change
         from options_df 
         where options_earliest_open>0.15
        group by date(time_converted), dte
        having max_options_pct_change = options_pct_change
        ), 
        
        MIN_TABLE AS (
        select distinct date(time_converted) as date
        , DTE_adjusted
        , day_classification
        , min(options_pct_change) as min_options_pct_change
        , time_converted as time_of_min_change
         from options_df 
         where options_earliest_open>0.15
        group by date(time_converted), dte
        having min_options_pct_change = options_pct_change
        )
        
        select 
        MAX_TABLE.date, MAX_TABLE.day_name, MAX_TABLE.DTE_adjusted, 
        MAX_TABLE.day_classification,MAX_TABLE.max_options_pct_change, MAX_TABLE.time_of_max_change
        , MIN_TABLE.min_options_pct_change, MIN_TABLE.time_of_min_change 
        from MAX_TABLE
        LEFT JOIN MIN_TABLE ON
        MAX_TABLE.date = MIN_TABLE.date 
        AND MAX_TABLE.DTE_adjusted = MIN_TABLE.DTE_adjusted
        """) 
max_min_df
#max_min_df.to_csv('max_min_stats_dte_adjusted.csv')

,date,day_name,DTE_adjusted,day_classification,max_options_pct_change,time_of_max_change,min_options_pct_change,time_of_min_change
0,2023-01-03,Tuesday,0,high,11.937500,2023-01-03 13:15:00.000000,0.003460,2023-01-03 15:45:00.000000
1,2023-01-03,Tuesday,1,high,5.588235,2023-01-03 13:00:00.000000,0.076923,2023-01-03 14:30:00.000000
2,2023-01-03,Tuesday,2,high,4.850000,2023-01-03 13:15:00.000000,0.147541,2023-01-03 14:30:00.000000
3,2023-01-04,Wednesday,0,average,2.705882,2023-01-04 12:45:00.000000,0.011765,2023-01-04 15:45:00.000000
4,2023-01-04,Wednesday,1,average,3.694444,2023-01-04 10:00:00.000000,0.150000,2023-01-04 15:45:00.000000
...,...,...,...,...,...,...,...,...
108,2023-02-27,Monday,1,average,1.875000,2023-02-27 10:15:00.000000,0.187500,2023-02-27 15:45:00.000000
109,2023-02-27,Monday,2,average,1.692308,2023-02-27 10:15:00.000000,0.277778,2023-02-27 15:45:00.000000
110,2023-02-28,Tuesday,0,very_low,1.520270,2023-02-28 13:30:00.000000,0.016667,2023-02-28 15:45:00.000000
111,2023-02-28,Tuesday,1,very_low,1.338028,2023-02-28 13:30:00.000000,0.296296,2023-02-28 13:30:00.000000


In [ ]:
max_min_df.head(10)

,date,day_name,DTE_adjusted,day_classification,max_options_pct_change,time_of_max_change,min_options_pct_change,time_of_min_change
0,2023-01-03,Tuesday,0,high,11.937500,2023-01-03 13:15:00.000000,0.003460,2023-01-03 15:45:00.000000
1,2023-01-03,Tuesday,1,high,5.588235,2023-01-03 13:00:00.000000,0.076923,2023-01-03 14:30:00.000000
2,2023-01-03,Tuesday,2,high,4.850000,2023-01-03 13:15:00.000000,0.147541,2023-01-03 14:30:00.000000
3,2023-01-04,Wednesday,0,average,2.705882,2023-01-04 12:45:00.000000,0.011765,2023-01-04 15:45:00.000000
4,2023-01-04,Wednesday,1,average,3.694444,2023-01-04 10:00:00.000000,0.150000,2023-01-04 15:45:00.000000
5,2023-01-04,Wednesday,2,average,1.862069,2023-01-04 12:45:00.000000,0.379310,2023-01-04 15:45:00.000000
6,2023-01-05,Thursday,0,very_low,1.411765,2023-01-05 14:45:00.000000,0.015873,2023-01-05 15:45:00.000000
7,2023-01-05,Thursday,1,very_low,1.284536,2023-01-05 13:30:00.000000,0.368421,2023-01-05 14:45:00.000000
8,2023-01-05,Thursday,2,very_low,1.264120,2023-01-05 14:45:00.000000,0.520833,2023-01-05 13:30:00.000000
9,2023-01-06,Friday,0,very_high,16.315789,2023-01-06 15:15:00.000000,0.001949,2023-01-06 15:45:00.000000


In [ ]:
# sqlite doesn't have pivot function, so now flip to pandas
# need to flip text fields to pandas native timestamps, otherwise pivot doesn't work 
max_min_df['time_of_max_change'] = pd.to_datetime(max_min_df['time_of_max_change'])
max_min_df['time_of_min_change'] = pd.to_datetime(max_min_df['time_of_min_change'])
# Create the pivot table
pivot_table = max_min_df.pivot_table(index=['date', 'day_name', 'day_classification'],
                                  columns='DTE_adjusted',
                                  values=['max_options_pct_change','time_of_max_change',
                                           'min_options_pct_change', 'time_of_min_change'])

print(pivot_table)
# pivot_table.to_csv('pivot_1_Jan03_Feb28_2023_dte_ad.csv')

                                        max_options_pct_change             \
DTE_adjusted                                                 0          1   
date       day_name  day_classification                                     
2023-01-03 Tuesday   high                            11.937500   5.588235   
2023-01-04 Wednesday average                          2.705882   3.694444   
2023-01-05 Thursday  very_low                         1.411765   1.284536   
2023-01-06 Friday    very_high                       16.315789   7.468750   
2023-01-09 Monday    average                          2.790698   2.125000   
2023-01-10 Tuesday   very_low                         1.593750   1.450000   
2023-01-11 Wednesday very_low                         1.535714   1.333333   
2023-01-12 Thursday  high                             2.956522   2.378049   
2023-01-13 Friday    high                             4.722222        NaN   
2023-01-17 Tuesday   very_low                         1.611111   1.367347   

In [ ]:
max_min_df['day_classification'].value_counts(normalize=True)

day_classification
high            0.327434
average         0.265487
very_low        0.230088
average_high    0.150442
very_high       0.026549
Name: proportion, dtype: float64

In [ ]:
max_min_df.groupby(['day_classification','DTE_adjusted']).agg(median_max_pct=('max_options_pct_change','median'))#.to_csv('agg_n1.csv')



median_max_pct
day_classification DTE_adjusted                
average            0                   2.095641
                   1                   1.805147
                   2                   1.671154
average_high       0                   2.602817
                   1                   2.220696
                   2                   2.155556
high               0                   5.208333
                   1                   3.882353
                   2                   3.545455
very_high          0                  16.315789
                   1                   7.468750
                   2                   6.500000
very_low           0                   1.535714
                   1                   1.378364
                   2                   1.334928

In [ ]:
# conn = sqlite3.connect('0-2DTE_spy_options_01Jan23-28Feb23v2_post_processed.db')
# options_df.to_sql('jan-feb23',conn, if_exists='replace', index=False)

124238